In [37]:
import numpy as np
import pandas as pd

#encoding module
import charset_normalizer

#plotting
import seaborn as sns
import matplotlib.pyplot as plt

#
import fuzzywuzzy
from fuzzywuzzy import process

np.random.seed(0)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [36]:
#read data
Professors = pd.read_csv('./pakistan_intellectual_capital.csv',encoding='utf-8')

In [20]:
Professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [26]:
Countries = Professors['Country'].unique()
Countries.sort()
print(Countries)

[' Germany' ' New Zealand' ' Sweden' ' USA' 'Australia' 'Austria' 'Canada'
 'China' 'Finland' 'France' 'Greece' 'HongKong' 'Ireland' 'Italy' 'Japan'
 'Macau' 'Malaysia' 'Mauritius' 'Netherland' 'New Zealand' 'Norway'
 'Pakistan' 'Portugal' 'Russian Federation' 'Saudi Arabia' 'Scotland'
 'Singapore' 'South Korea' 'SouthKorea' 'Spain' 'Sweden' 'Thailand'
 'Turkey' 'UK' 'USA' 'USofA' 'Urbana' 'germany']


#### Inconsistencies:
Same country name: Germany/germany  -  Extra spaces at the begining and end of country names - Extra space in the middle of the name (southKorea vs south korea)

In [30]:
## What we can do:
# Make are letters lower case
Professors['Country'] = Professors['Country'].str.lower()
# Remove the trailing white spaces
Professors['Country'] = Professors['Country'].str.strip()

In [32]:
print(Professors['Country'].unique())

['thailand' 'pakistan' 'germany' 'austria' 'australia' 'uk' 'china'
 'france' 'usofa' 'southkorea' 'malaysia' 'sweden' 'italy' 'canada'
 'norway' 'ireland' 'new zealand' 'urbana' 'portugal' 'russian federation'
 'usa' 'finland' 'netherland' 'greece' 'turkey' 'south korea' 'macau'
 'singapore' 'spain' 'japan' 'hongkong' 'saudi arabia' 'mauritius'
 'scotland']


## fuzzywuzzy

For small data sets we can corrcet the inconsistencies with hand. But we cannot do so for larger data sets

We're going to use the fuzzywuzzy package to help identify which strings are closest to each other.

Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings.

In [40]:
#Here, we're going to get the ten strings from our list of cities that have the closest distance to "south korea".
matches = fuzzywuzzy.process.extract('south korea', Countries,limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('South Korea', 100),
 ('SouthKorea', 48),
 ('Saudi Arabia', 43),
 ('Norway', 35),
 ('Ireland', 33),
 ('Portugal', 32),
 ('Singapore', 30),
 ('Netherland', 29),
 ('Macau', 25),
 ('USofA', 25)]

In [48]:
# Let's replace all rows in our "Country" column that have a ratio of > 47 with "south korea".
def replace_matches_in_column(df,column, str_to_match, min_ratio=47):
    str_column = df[column].unique()
    matches = fuzzywuzzy.process.extract(str_to_match,str_column, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    #only get close matches with ratio>=miin_ratio
    close_match = [matches[0] for matches in matches if matches[1]>=min_ratio]
    #get the row number for close_match
    row_close_match = df[column].isin(close_match)
    #replace close_match strings with the str_to_match
    df.loc[row_close_match,column] = str_to_match
    print("All Done!")

replace_matches_in_column(df=Professors,column='Country',str_to_match='south korea')
replace_matches_in_column(df=Professors,column='Country',str_to_match='New Zealand')

All Done!
All Done!


In [49]:
Professors['Country'].unique()

array(['New Zealand', 'Pakistan', 'germany', 'Austria', 'Australia', 'UK',
       'China', 'France', 'USofA', 'south korea', 'Malaysia', 'Sweden',
       'Italy', 'Canada', 'Norway', 'Urbana', 'Portugal',
       'Russian Federation', 'USA', ' USA', ' Sweden', 'Greece', 'Turkey',
       'Macau', 'Singapore', 'Spain', 'Japan', 'HongKong', 'Saudi Arabia',
       'Mauritius', 'Scotland'], dtype=object)